In [ ]:
pip install transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 15.7 MB/s eta 0:00:00


In [ ]:
documents = [
    "The capital of France is Paris.",
    "The Eiffel Tower is located in Paris.",
    "The Louvre Museum is the world's largest art museum.",
    "Paris is known for its cafe culture and landmarks."
]

In [ ]:
import faiss
from transformers import BertTokenizer, BertModel
import numpy as np

# 문서 임베딩 생성 함수
def get_embeddings(texts, model, tokenizer):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.detach().numpy()

# BERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# 문서 임베딩 생성
doc_embeddings = get_embeddings(documents, model, tokenizer)

# FAISS 인덱스 생성
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

# 검색 함수
def search(query, top_k=1):
    query_embedding = get_embeddings([query], model, tokenizer)
    D, I = index.search(query_embedding, top_k)
    return [documents[i] for i in I[0]]

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# GPT-2 모델과 토크나이저 로드
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')

# 답변 생성 함수
def generate_answer(query, context):
    input_text = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    inputs = gpt2_tokenizer.encode(input_text, return_tensors='pt')
    outputs = gpt2_model.generate(inputs, max_length=100, num_return_sequences=1)
    return gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# 사용자 질문
query = "What is the capital of France?"

# 관련 문서 검색
retrieved_docs = search(query, top_k=1)
context = retrieved_docs[0]

# 답변 생성
answer = generate_answer(query, context)
print(f"Question: {query}")
print(f"Answer: {answer}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: What is the capital of France?
Answer: Context: The capital of France is Paris.

Question: What is the capital of France?

Answer: The capital of France is Paris.

Question: What is the capital of France?

Answer: The capital of France is Paris.

Question: What is the capital of France?

Answer: The capital of France is Paris.

Question: What is the capital of France?

Answer: The capital of France is Paris.

Question
